# 导入paddle包

In [2]:
import os
import paddle.v2 as paddle
import paddle.v2.dataset.uci_housing as uci_housing

In [3]:
with_gpu = os.getenv('WITH_GPU', '0') != '0'

# 初始化paddle

In [4]:
paddle.init(use_gpu=with_gpu, trainer_count=1)

# 构造模型

In [5]:
x = paddle.layer.data(name='x', type=paddle.data_type.dense_vector(13))
y_predict = paddle.layer.fc(input=x, size=1, act=paddle.activation.Linear())
y = paddle.layer.data(name='y', type=paddle.data_type.dense_vector(1))

# 目标函数

In [6]:
cost = paddle.layer.square_error_cost(input=y_predict, label=y)

# 保存模型拓扑

In [7]:
inference_topology = paddle.topology.Topology(layers=y_predict)
with open("inference_topology.pkl", 'wb') as f:
    inference_topology.serialize_for_inference(f)

# 构造参数

In [42]:
parameters = paddle.parameters.create(cost)
print parameters.keys()

[u'___fc_layer_0__.w0', u'___fc_layer_0__.wbias']
[u'___fc_layer_0__.w0', u'___fc_layer_0__.wbias']


# 优化方法

In [19]:
optimizer = paddle.optimizer.Momentum(momentum=0)

In [20]:
trainer = paddle.trainer.SGD(
        cost=cost, parameters=parameters, update_equation=optimizer)

# 定义回调函数

In [21]:
feeding = {'x': 0, 'y': 1}

In [22]:
    def event_handler(event):
        if isinstance(event, paddle.event.EndIteration):
            if event.batch_id % 100 == 0:
                print "Pass %d, Batch %d, Cost %f" % (
                    event.pass_id, event.batch_id, event.cost)

        if isinstance(event, paddle.event.EndPass):
            if event.pass_id % 10 == 0:
                with open('params_pass_%d.tar' % event.pass_id, 'w') as f:
                    trainer.save_parameter_to_tar(f)
            result = trainer.test(
                reader=paddle.batch(uci_housing.test(), batch_size=2),
                feeding=feeding)
            print "Test %d, Cost %f" % (event.pass_id, result.cost)

# 模型训练

In [28]:
trainer.train(
        reader=paddle.batch(
            paddle.reader.shuffle(uci_housing.train(), buf_size=5000),
            batch_size=2),
        feeding=feeding,
        event_handler=event_handler,
        num_passes=100)

Pass 0, Batch 0, Cost 1.495800
Pass 0, Batch 100, Cost 42.483418
Pass 0, Batch 200, Cost 26.257704
Test 0, Cost 25.402659
Pass 1, Batch 0, Cost 3.716550
Pass 1, Batch 100, Cost 2.476472
Pass 1, Batch 200, Cost 7.149185
Test 1, Cost 30.745138
Pass 2, Batch 0, Cost 27.800522
Pass 2, Batch 100, Cost 101.234329
Pass 2, Batch 200, Cost 9.057484
Test 2, Cost 29.210953
Pass 3, Batch 0, Cost 14.587635
Pass 3, Batch 100, Cost 4.292050
Pass 3, Batch 200, Cost 10.290013
Test 3, Cost 27.876014
Pass 4, Batch 0, Cost 1.026340
Pass 4, Batch 100, Cost 4.946856
Pass 4, Batch 200, Cost 45.496811
Test 4, Cost 24.579970
Pass 5, Batch 0, Cost 4.527252
Pass 5, Batch 100, Cost 41.499603
Pass 5, Batch 200, Cost 2.898273
Test 5, Cost 29.591355
Pass 6, Batch 0, Cost 1.956164
Pass 6, Batch 100, Cost 32.670002
Pass 6, Batch 200, Cost 2.286242
Test 6, Cost 25.374574
Pass 7, Batch 0, Cost 10.573087
Pass 7, Batch 100, Cost 10.262197
Pass 7, Batch 200, Cost 6.114974
Test 7, Cost 22.367839
Pass 8, Batch 0, Cost 4.0463

Pass 68, Batch 100, Cost 16.610296
Pass 68, Batch 200, Cost 15.630659
Test 68, Cost 33.901514
Pass 69, Batch 0, Cost 21.351261
Pass 69, Batch 100, Cost 2.626736
Pass 69, Batch 200, Cost 1.238070
Test 69, Cost 31.455544
Pass 70, Batch 0, Cost 13.155558
Pass 70, Batch 100, Cost 22.029736
Pass 70, Batch 200, Cost 27.927612
Test 70, Cost 31.331099
Pass 71, Batch 0, Cost 0.299671
Pass 71, Batch 100, Cost 6.401699
Pass 71, Batch 200, Cost 36.608990
Test 71, Cost 27.742521
Pass 72, Batch 0, Cost 11.845661
Pass 72, Batch 100, Cost 5.463381
Pass 72, Batch 200, Cost 34.234097
Test 72, Cost 32.939042
Pass 73, Batch 0, Cost 0.699001
Pass 73, Batch 100, Cost 37.642380
Pass 73, Batch 200, Cost 10.266459
Test 73, Cost 31.769186
Pass 74, Batch 0, Cost 20.275169
Pass 74, Batch 100, Cost 27.244501
Pass 74, Batch 200, Cost 49.338493
Test 74, Cost 31.083532
Pass 75, Batch 0, Cost 16.081484
Pass 75, Batch 100, Cost 1.898904
Pass 75, Batch 200, Cost 3.906583
Test 75, Cost 30.027385
Pass 76, Batch 0, Cost 6.

# 预测

In [29]:
test_data_creator = paddle.dataset.uci_housing.test()
test_data = []
test_label = []

In [30]:
for item in test_data_creator():
    test_data.append((item[0], ))
    test_label.append(item[1])
    if len(test_data) == 5:
        break

In [31]:
probs = paddle.infer(output_layer=y_predict, parameters=parameters, input=test_data)

In [32]:
for i in xrange(len(probs)):
    print "label=" + str(test_label[i][0]) + ", predict=" + str(probs[i][0])

label=8.5, predict=10.135715
label=5.0, predict=12.904484
label=11.9, predict=8.694727
label=27.9, predict=20.11639
label=17.2, predict=12.672748


# 练习-降低cost